In [ ]:
# ==============================================================================
# БЛОК 1: ИМПОРТ БИБЛИОТЕК И ОБЪЯВЛЕНИЕ ФУНКЦИЙ
# ==============================================================================

import pandas as pd
import re
from IPython.display import display

def get_industry(title, journal):
    """
    Классифицирует статью по одной из трех отраслей на основе ключевых слов
    в названии и журнале.
    """
    title_journal_lower = (str(title) + ' ' + str(journal)).lower()

    it_keywords = [
        'computer', 'software', ' ml ', 'machine learning', 'artificial intelligence',
        'gnn', 'recommender', 'supercomputing', 'kv stores', 'informatics', 'data',
        'algorithm', 'internet', 'programming', 'heuristics', 'ieee',
        'vision and applications', 'applied intelligence', 'online learning',
        'information theory', 'robot-assisted', 'robotic'
    ]

    econ_keywords = [
        'economic', 'finance', 'trade', 'poverty', 'development', 'policy',
        'management', 'market', 'investment', 'sdgs', 'tourism', 'transport',
        'financial', 'reimbursement', 'inclusion', 'business', 'corporate',
        'wealth', 'econometric', 'socio-economic', '-economic', 'geographer',
        'promotion management', 'lifestyle and sdgs', 'review of income',
        'risk management', 'sustainability', 'sustainable'
    ]

    bio_keywords = [
        'metabolomics', 'biological', 'health', 'medical', 'nursing', 'paediatric',
        'clinical', 'vaccination', 'medicine', 'neurosurgery', 'anaesthesiologica',
        'nutrition', 'cancer', 'palliative', 'hernia', 'care', 'urolog', 'gynecology',
        'geriatric', 'psychology', 'adhd', 'bereavement', 'stroke', 'surgery',
        'diabetes', 'bioscience', 'biomarkers', 'biomolecules', 'food', 'seaweed'
    ]

    if any(keyword in title_journal_lower for keyword in it_keywords):
        return 'Информатика'
    if any(keyword in title_journal_lower for keyword in econ_keywords):
        return 'Экономика'
    if any(keyword in title_journal_lower for keyword in bio_keywords):
        return 'Биоинформатика'

    return 'Прочее'

In [ ]:
# ==============================================================================
# БЛОК 2: ОСНОВНАЯ ЛОГИКА СКРИПТА (ИСПРАВЛЕННАЯ ВЕРСИЯ)
# ==============================================================================

file_path = 'eLibrary.txt'
try:
    with open(file_path, 'r', encoding='utf-8') as file:
        text_data = file.read()
    print(f"✅ Файл '{file_path}' успешно прочитан.")
except FileNotFoundError:
    print(f"❌ Ошибка: Файл '{file_path}' не найден!")
    print("➡️ Пожалуйста, загрузите файл в рабочую директорию и запустите ячейку снова.")
    text_data = None

if text_data:
    entries = re.split(r'\n(?=\d+\n)', text_data.strip())
    parsed_data = []
    print(f"Найдено {len(entries)} записей для обработки.")

    for entry in entries:
        if not entry.strip():
            continue

        lines = [line.strip() for line in entry.strip().split('\n') if line.strip()]

        if len(lines) < 4:
            print(f"⚠️ Предупреждение: Пропускается некорректная запись: {' '.join(lines)}")
            continue

        try:
            # ---  ЛОГИКА ПАРСИНГА ---

            record_id = int(lines[0])

            # Последняя строка содержит и журнал, и цитирования. Разделяем их.
            last_line = lines[-1]
            if '\t' in last_line:
                journal_info_line, citations_str = last_line.rsplit('\t', 1)
            else:
                journal_info_line, citations_str = last_line.rsplit(' ', 1) # Резервный вариант

            citations_rsci = int(citations_str.strip())

            # Строка перед последней — это авторы.
            authors = lines[-2]

            # Все, что между ID и авторами, — это заголовок.
            title = " ".join(lines[1:-2])
            title = re.sub(r"^(Полный текст доступен на внешнем сайте|Можно приобрести полный текст этой публикации за \d+ руб\.)", "", title).strip()

            # Парсим информацию о журнале, как и раньше
            year_match = re.search(r'\b(20\d{2})\b', journal_info_line)
            year = int(year_match.group(1)) if year_match else None
            journal_name = journal_info_line.split('.')[0].strip()

            industry = get_industry(title, journal_name)

            parsed_data.append({
                'ID': record_id,
                'Title': title,
                'Authors': authors,
                'Journal': journal_name,
                'Year': year,
                'Citations_RSCI': citations_rsci,
                'Industry': industry,
                'Source': 'eLibrary'
            })
        except (ValueError, IndexError) as e:
            print(f"❌ Ошибка при обработке записи c ID {lines[0]}: {e}\n   Запись: '{' '.join(lines[:2])}...'\n")

    if parsed_data:
        df = pd.DataFrame(parsed_data)
        csv_filename = 'dataset1.csv'
        excel_filename = 'dataset1.xlsx'

        df.to_csv(csv_filename, index=False, encoding='utf-8-sig')
        df.to_excel(excel_filename, index=False)

        print(f"\n✅ Данные успешно обработаны и сохранены в файлы '{csv_filename}' и '{excel_filename}'.")
        print("\n📊 Первые 10 строк итоговой таблицы:")
        display(df.head(10))
    else:
        print("\n⚠️ В файле не найдено данных для обработки.")